### 11장. 심층 신경망 훈련하기.

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 
import numpy as np 

import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

2021-10-18 18:50:35.028811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 18:50:35.028856: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


- keras는 기본적으로, 균등분포의 글로럿 초기화를 사용합니다. 다음과 같이 층을 만들 때,
kernel_initializer="he_uniform"이나 kernel_initializer="he_normal"로 바꾸어
He 초기화를 사용할 수 있습니다.

- 불안정한 Gradient 폭주, 혹은 감소를 막고 안정적인 전달을 위해
- 역방향에서 층을 통과하기 전, 후의 Gradient 분산이 동일해야한다.
- 글로럿 초기화

In [3]:
keras.layers.Dense(10,activation="relu",kernel_initializer="he_normal")

#### 수렴하지 않는 활성화 함수
- 활성화 함수를 잘못 선택하면, 자칫 Gradient loss 혹은 Gradient explosion으로 이어질 수 있다는 우려가 발생한다.
- 그전에는 단지 시그모이드 함수가 최선일 거라 생각했으나, 다른 활성화 함수가 심층 신경망에서 훨씬 더 잘 작동한다는 사실이 발견되었습니다.
-특히 Relu의 경우 하습이 진행됨에 따라 가중치 합이 음수가 되어 -> Gradient가 0이 되므로
- 경사하강법이 더 이상 작동하지 않게 된다.

- 이 문제를 해결하기 위해 LeakyReLU와 같은 Relu함수의 변종을 사용합니다. 
- RReLU나 PReLU도 있음..

#### 여러가지 활성화 함수를 쓰는 방법
- LeakyRelu 층을 만들고, 모델에서 적용하려는 층 뒤에 추가합니다. 
- keras.layers.LeakyReLU(alpha=0.2)

- SELU 활성화 함수를 사용하려면 다음과 같습니다.

In [4]:
layer = keras.layers.Dense(10,activation="selu", kernel_initializer="lecun_normal")


### 배치 정규화
- 위와 같은 변종 활성화 함수를 사용하면 훈련 초기단계에서는 감속 / 폭주를 막을 수 있다.
- 하지만 훈련하는 동안 다시 발생하지 않으리란 보장은 없습니다. 
- 이에 따라 배치 정규화(batch normalization) 기법을 제안합니다. 
- 이게 뭐냐하면, 각 층에서 활성화 함수를 통과하기 전이나, 후에, 모델에 연산을 하나 추가하는 것입니다.

- 그러니까, 들어온 m개의 입력 데이터에 대한 정규화를 한다는 소리는,
- 입력 데이터마다 하나의 평균을 가지고, 그에 따른 표준편차 벡터를 구합니다.
- 이를 기반으로 각 x벡터에 평균을 제한 값 / 표준편차루트+안전항 을 통해 나온 새로운 x벡터를 
- 각 파라미터 벡터에 곱한 뒤 출력 이동 벡터를 더하여 
- 새로운 z를 만들어낸 배치 정규화 연산 값입니다.

- 하튼 걍 표준화를 하고 1차함수 하나를 통과시키는 과정을 뜻합니다.

#### BatchNormalization 층 
- 

In [7]:
model = keras.models.Sequential([ 
    keras.layers.Flatten(input_shape=[28,28]), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300,activation="elu",kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100,activation="elu",kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(10,activation="softmax")
])

2021-10-18 18:50:57.707742: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 18:50:57.707902: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-18 18:50:57.707974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-18 18:50:57.711189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-10-18 18:50:57.711260: W tensorflow/stream_executor/platform/default/dso_loader

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 784)               3136      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 300)               1200      
_________________________________________________________________
dense_5 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1

In [9]:
[(var.name,var.trainable) for var in model.layers[1].variables]

[('batch_normalization_2/gamma:0', True),
 ('batch_normalization_2/beta:0', True),
 ('batch_normalization_2/moving_mean:0', False),
 ('batch_normalization_2/moving_variance:0', False)]

In [10]:
model = keras.models.Sequential([ 
    keras.layers.Flatten(input_shape=[28,28]), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300,kernel_initializer="he_normal",use_bias=False), 
    keras.layers.BatchNormalization(), 
    keras.layers.Activation("elu"), 
    keras.layers.Dense(100,kernel_initializer="he_normal",use_bias=False), 
    keras.layers.BatchNormalization(), 
    keras.layers.Activation("elu"), 
    keras.layers.Dense(10,activation="softmax")
])

### 그레이디언트 클리핑
- Gradient 폭주를 완화하는 방법. 역전파 될 때 일정 임곗값을 넘어서지 못하게 Gradient를 잘라내는 것입니다.
- 케라스에서 이를 구현하려면, 다음과 같이 옵티마이저를 만들 때 clipvalue와 clipnorm 매개변수를 지정하면 됩니다.

In [12]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse",optimizer=optimizer)

- 이 옵티마이저는 Gradient vector의 모든 원소를 -1.0과 1.0 사이로 클리핑합니다. 
- 즉 모든 손실의 편미분 값을 -1.0과 1.0 사이로 잘라냅니다. 임곗값은 하이퍼파라미터로 튜닝할 수 있습니다.

### 사전 훈련된 층을 재사용하기
- 재사용한다. 
- 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아본 다음
- 그 신경망의 하위층을 재사용하는 것이 좋습니다.

#### 이를 전이 학습(transfer learning)이라고 합니다. 
- 이 방법은 훈련 속도를 크게 높일 뿐만 아니라, 필요한 훈련 데이터도 크게 줄여줍니다. 
- 비슷한 점이 많고 심지어 일부 겹치기도 하므로 N-1번째 신경망의 일부를 재사용해봐야 합니다.

In [15]:
model_A = keras.models.load_model()

TypeError: load_model() missing 1 required positional argument: 'filepath'